In [1]:
import os, glob

In [2]:
import pandas as pd
import geopandas as gpd
import rioxarray as rioxr

In [3]:
import dask

In [4]:
from pyce import mnt, shape, sheds

# 1. Lidar HD

In [ ]:
home_dir = "/home/aguerou/CARRTEL/lacs_sentinelles/ls_bv_lc/data/main_lakes/"

In [ ]:
dwnld_dir = os.path.join(home_dir, "dem/dwnld_tiles/")

## 1.1 Identify lidar HD tiles

In [ ]:
gdf_bv = gpd.read_file(os.path.join(home_dir, "bv/bv_convexHull.shp"))

In [ ]:
gdf_lidar = gpd.read_file(os.path.join(home_dir, "../grid_lidar_hd/TA_diff_pkk_lidarhd_classe.shp"))

In [ ]:
gdf_lakes_lidar = mnt.identify_lidarhd_tiles(df=gdf_bv, 
                                             gdf_lidar=gdf_lidar,
                                             buffer=0, 
                                             plot=True)                                       

In [ ]:
len(set(gdf_lakes_lidar["nom_pkk"]))

## 1.2 Save list of files

In [ ]:
gdf_to_save = gdf_lakes_lidar.drop(columns=["geometry"])
gdf_to_save["dir_tile_dwnld"] = dwnld_dir
gdf_to_save["tile_name"] = gdf_to_save["nom_pkk"].apply(lambda x: mnt._get_lidar_basename(x))
gdf_to_save = gdf_to_save[["lake_name", "tile_name", "dir_tile_dwnld", "url_telech"]]

In [ ]:
gdf_to_save.to_csv(os.path.join(home_dir, "dem/lacsSentinelles_list_MNT_IGN_lidarHD_202409.csv"), index=False)

## 1.3 Download lidarHD tiles

In [ ]:
mnt.download_lidarhd_tiles(gdf_lakes_lidar,
                           dwnld_dir=dwnld_dir, 
                           force=False,
                           num_workers=10,
                           compute=True)

## 1.4 Process lidar data

### 1.4.1. Special lakes

Some lakes are overlapping over multiple corners of lidar HD tiles. 

Lidar data being disturbed on water surface, one needs to merge such tiles to allow triangulation to work by merging such tiles two by two.

One need to change the associated names in the dataframe file list to process afterwards

In [ ]:
gdf_lidar = pd.read_csv(os.path.join(home_dir, "dem/lacsSentinelles_list_MNT_IGN_lidarHD_202409.csv"))

#### Merlet

In [ ]:
gdf_merlet = gdf_lidar.loc[gdf_lidar.lake_name == "Merlet superieur"]

In [ ]:
merlet_to_merge = [os.path.join(dwnld_dir, tile_name)+".copc.laz" for tile_name in gdf_merlet["tile_name"]]
merlet_to_merge.sort()

In [ ]:
# Turn on compute to process - here just to check names
# -----------------------------------------------------
# WARNING: one needs after computation to rename output file manually to .copc.laz instead of .laz
# The .laz extension is mandatory for the qgis merging command to work. 
# One could also add this renaming inside the mnt.merge_lidarhd_tiles function - not done  yet
merlet_f1 = mnt.merge_lidarhd_tiles(file_list=merlet_to_merge[:2], save_dir=dwnld_dir, compute=False) 

In [ ]:
merlet_f2 = mnt.merge_lidarhd_tiles(file_list=merlet_to_merge[2:], save_dir=dwnld_dir, compute=False)

Replace tiles name

In [ ]:
gdf_lidar.loc[gdf_lidar.lake_name=="Merlet superieur"]

In [ ]:
gdf_lidar.iloc[244]["tile_name"] = os.path.splitext(os.path.basename(merlet_f1))[0]
gdf_lidar.iloc[245]["tile_name"] = os.path.splitext(os.path.basename(merlet_f2))[0]

In [ ]:
gdf_lidar.drop(index=[246,247], inplace=True)
gdf_lidar.reset_index(drop=True, inplace=True)

#### Cornu

In [ ]:
gdf_cornu = gdf_lidar.loc[gdf_lidar.lake_name == "Cornu"]

In [ ]:
cornu_to_merge = [os.path.join(dwnld_dir, tile_name)+".copc.laz" for tile_name in gdf_cornu["tile_name"]]
cornu_to_merge.sort()

In [ ]:
# Turn on compute to process - here just to check names
# -----------------------------------------------------
# WARNING: one needs after computation to rename output file manually to .copc.laz instead of .laz
# The .laz extension is mandatory for the qgis merging command to work. 
# One could also add this renaming inside the mnt.merge_lidarhd_tiles function - not done  yet
cornu_f1 = mnt.merge_lidarhd_tiles(file_list=cornu_to_merge[:2], save_dir=dwnld_dir, compute=False)

In [ ]:
cornu_f2 = mnt.merge_lidarhd_tiles(file_list=cornu_to_merge[2:], save_dir=dwnld_dir, compute=False)

Replace tiles name

In [ ]:
gdf_lidar.loc[gdf_lidar.lake_name=="Cornu"]

In [ ]:
gdf_lidar.iloc[157]["tile_name"] = os.path.splitext(os.path.basename(cornu_f1))[0]
gdf_lidar.iloc[158]["tile_name"] = os.path.splitext(os.path.basename(cornu_f2))[0]

In [ ]:
gdf_lidar.drop(index=[159,160], inplace=True)
gdf_lidar.reset_index(drop=True, inplace=True)

### 1.4.2 Save dataframe with merged tiles

In [ ]:
gdf_lidar.to_csv(os.path.join(home_dir, "dem/lacsSentinelles_list_MNT_IGN_lidarHD_202409_merged.csv"), index=False)

### 1.4.3 Processing (triangulation)

In [ ]:
gdf_lidar = pd.read_csv(os.path.join(home_dir, "dem/lacsSentinelles_list_MNT_IGN_lidarHD_202409_merged.csv"))

In [ ]:
process_dir = os.path.join(home_dir, "dem/process_tiles/")
resolution = 1

Check how many tiles left to process

In [ ]:
lidar_files_tif = [os.path.join(process_dir, tile_name)+".tif" for tile_name in gdf_lidar["tile_name"]]

In [ ]:
count=0
for file in lidar_files_tif:
    if not os.path.exists(file): 
        count+=1
print(f"Number of tiles to process: {count}")

Processing

In [ ]:
lidar_files = [os.path.join(tile_dir, tile_name)+".copc.laz" for tile_dir, tile_name in zip(gdf_lidar["dir_tile_dwnld"], gdf_lidar["tile_name"])]

In [ ]:
%%time
mnt.process_lidarhd_tiles(lidar_files, 
                          save_dir=process_dir, 
                          resolution=resolution, 
                          num_workers=2, 
                          redo=False)

## 1.5 Merge DEM tiles by lake

In [ ]:
df_lakes = pd.read_csv(os.path.join(home_dir, "dem/lacsSentinelles_list_MNT_IGN_lidarHD_202409_merged.csv"))

In [ ]:
df_lakes.loc[:, "dir_tile_process"] = os.path.join(home_dir, "dem/process_tiles/")

In [ ]:
%%time
for lake in set(df_lakes.lake_name):
    print(lake)
    df = df_lakes.loc[df_lakes.lake_name == lake]
    dem_merged = mnt.merge_dem_tiles(dem=df, 
                                     df_metadata = dict(col_name_dir="dir_tile_process",
                                                        col_name_tile="tile_name", 
                                                        tile_extension=".tif"),
                                     epsg_mnt="epsg:2154",
                                     save_name=os.path.join(home_dir, f"dem/dem_1m/{lake.replace(' ', '_').lower()}.tif")

# 2. Get sheds

In [ ]:
home_dir = "/home/aguerou/CARRTEL/lacs_sentinelles/ls_bv_lc/data/main_lakes/"

## 2.1  - Coordinates file

* Deal with coordinates files vs. centers and outlets shapefiles
* Here the coordinates (centers and outlets) are generated through QGIS & then saved as csv files
* See also section 2.4.1 to update outlet shapefile and csv file when deriving sheds

In [1]:
center_lakes = gpd.read_file(os.path.join(home_dir, "coords/lacsSentinelles_centers_202409.shp")).sort_values(by="lake_name").reset_index(drop=True)
outlet_lakes = gpd.read_file(os.path.join(home_dir, "coords/lacsSentinelles_outlets_202409.shp")).sort_values(by="lake_name").reset_index(drop=True)

NameError: name 'gpd' is not defined

In [ ]:
# Check if centers/outlets have the same list and order of lakes
all(center_lakes["lake_name"] == outlet_lakes["lake_name"])

In [ ]:
coords_lakes = pd.DataFrame()

coords_lakes["lake_name"] = outlet_lakes.lake_name

coords_lakes["proj_center"] = center_lakes.crs.to_string()
coords_lakes["x_center"] = center_lakes.geometry.x
coords_lakes["y_center"] = center_lakes.geometry.y

coords_lakes["proj_outlet"] = outlet_lakes.crs.to_string()
coords_lakes["x_outlet"] = outlet_lakes.geometry.x
coords_lakes["y_outlet"] = outlet_lakes.geometry.y

In [ ]:
coords_lakes.to_csv(os.path.join(home_dir, "coords/lacsSentinelles_coords_202409.csv"), index=False)

## 2.2 Get lake shapes

* To derive a watershed, one needs to flatten and burn flat parts of a DEM, sush as lakes
* Here we identify lakes countours (i.e., lake shapes) from the DEM ahead of the water shed estimation
* We identify such regions based on the altitude of lakes central point +/- an elevation threshold. 
* Everything within these elevations are considered as lakes, thresholds depend on the lake and are evaluated empirically
* Manual editing is however still needed on QGIS afterwards

In [ ]:
df_lakes = pd.read_csv(os.path.join(home_dir, "coords/lacsSentinelles_coords_202409.csv"))

### Specify flatenning threshold

In [ ]:
df_lakes["thresh"] = 0.3 # in meters

In [ ]:
lake_sel = ["Blanc du Carro", "Lauzanier", "Pisses", "Aumar", "Bresses inferieur", "Bresses superieur"]
lake_sel_low = ["Nino"]
lake_sel_high = ["Corne"]
lake_sel_very_high = ["Izourt"]

In [ ]:
df_lakes.loc[df_lakes.lake_name.isin(lake_sel), "thresh"] = 0.1
df_lakes.loc[df_lakes.lake_name.isin(lake_sel_low), "thresh"] = 0.05
df_lakes.loc[df_lakes.lake_name.isin(lake_sel_high), "thresh"] = 5
df_lakes.loc[df_lakes.lake_name.isin(lake_sel_very_high), "thresh"] = 7

### Run lake shape

In [ ]:
list_lakes = list(set(df_lakes["lake_name"]))

In [ ]:
gdf_lake = []

for lake in list_lakes:
    dem = rioxr.open_rasterio(os.path.join(home_dir, f"dem/dem_1m/{lake.replace(' ', '_')}.tif"))
                              
    d_lake = df_lakes.loc[df_lakes.lake_name == lake]
    
    lake_shp = sheds.run_get_lake_shape(dem=dem, 
                                        xy_lake=[d_lake["x_center"].values[0], 
                                                 d_lake["y_center"].values[0]],
                                        crs_lake=d_lake["proj_center"].values[0],
                                        flattening_thresh=d_lake["thresh"].values[0],
                                        name_lake=lake,
                                        as_gdf=True)  
    gdf_lake.append(lake_shp)

In [ ]:
%%time
gdf_lake = dask.compute(gdf_lake, num_workers=4)

In [ ]:
gdf_lake = pd.concat([gdf_lake[0][n] for n in range(len(gdf_lake[0]))]).sort_values(by="lake_name").reset_index(drop=True)

In [ ]:
gdf_lake.to_file(os.path.join(home_dir, "bv/lacsSentinelles_lakes_auto_202409.shp"))

## 2.3 Burn lake shapes

!!! After manual editing !!!
* Editing needed on QGIS to clean lake shapes

* IMPORTANT : 
    * One also needs to delineate manually other lakes within potential water sheds to improve results
    * I personnally added them with the following naming within the orignal shape file: 'lake_Lxx' with xx being increasing numbers   
    * Final file is:  
        * "lacsSentinelles_lakes_202409.shp"

### Get data

In [ ]:
df_lakes = pd.read_csv(os.path.join(home_dir, "coords/lacsSentinelles_coords_202409.csv"))

In [ ]:
lake_shapes = gpd.read_file(os.path.join(home_dir, "bv/lacsSentinelles_lakes_202409.shp"))

### On 1m data

In [ ]:
list_lakes = list(set(df_lakes["lake_name"]))

In [ ]:
%%time
for lake in list_lakes:
    dem = rioxr.open_rasterio(os.path.join(home_dir, f"dem/dem_1m/{lake.replace(' ', '_')}.tif"))
    lake_shp = lake_shapes.loc[lake_shapes["lake_name"] == lake]

    dem_burnt = sheds.burn_lake_to_dem(dem=dem, 
                                       lake_shp=lake_shp, 
                                       method="min",
                                       save_name=os.path.join(home_dir, f"dem/dem_1m/{lake.replace(' ', '_')}_burnt.tif"))

## 2.4 Get sheds

#### Get data + run

In [ ]:
df_lakes = pd.read_csv(os.path.join(home_dir, "coords/lacsSentinelles_coords_202409.csv"))

In [ ]:
list_lakes = list(set(df_lakes["lake_name"]))

In [ ]:
gdf_shed = []

for lake in list_lakes:
    
    dem = rioxr.open_rasterio(os.path.join(home_dir, f"dem/dem_1m/{lake.replace(' ', '_')}_burnt.tif"))
    
    d_lake = df_lakes.loc[df_lakes["lake_name"] == lake]
                              
    shed = sheds.run_get_shed(dem=dem,
                              xy_outlet=[d_lake["x_outlet"].values[0],
                                         d_lake["y_outlet"].values[0]],
                              crs_outlet=d_lake["proj_outlet"].values[0],
                              name_shed=lake,
                              buffer_size=5, #in unit of crs / to clean resulting shed shape (merging multipolygons)
                              buffer_delta=0, #in unit of crs / to clean resulting shed shape (merging multipolygons)
                              save_acc=os.path.join(home_dir, f"bv/acc_1m/{lake.replace(' ', '_')}_acc.tif"))
    gdf_shed.append(shed)

In [ ]:
%%time
gdf_shed = dask.compute(gdf_shed, num_workers=4)

#### Save sheds + snap

In [ ]:
gdf_shed = pd.concat([gdf_shed[0][n] for n in range(len(gdf_shed[0]))]).sort_values(by="lake_name").reset_index(drop=True)

In [ ]:
gdf_shed.to_file(os.path.join(home_dir, "bv/lacsSentinelles_sheds_202409.shp"))

In [ ]:
shape.df_to_gdf(gdf_shed[["lake_name","x_snap","y_snap"]],
                x_col="x_snap",
                y_col="y_snap",
                crs=d_lake["proj_outlet"].values[0]
               ).to_file(os.path.join(home_dir, "bv/lacsSentinelles_snap_202409.shp"))

#### Update outlets from shape to file
!Only if modified in QGIS!
* This is needed to ensure correct estimation of the water shed
* Move the outlet position so the shed and the snap point correspond well to the true outlet of the lake
* To be checked on QGIS, and modified if necessary 
* Then re-run the water shed estimation AFTER having modified the coords file here below

In [ ]:
gdf_outlet = gpd.read_file(os.path.join(home_dir, "coords/lacsSentinelles_outlets_202409.shp"))

In [ ]:
df_lakes = pd.read_csv(os.path.join(home_dir, "coords/lacsSentinelles_coords_202409.csv"))

In [ ]:
df_lakes["x_outlet"] = gdf_outlet["geometry"].x
df_lakes["y_outlet"] = gdf_outlet["geometry"].y

In [ ]:
df_lakes.to_csv(os.path.join(home_dir, "coords/lacsSentinelles_coords_202409.csv"), index=False)

## 2.5 Remove lake from sheds

### Data

In [ ]:
gdf_sheds = gpd.read_file(os.path.join(home_dir, "bv/lacsSentinelles_sheds_202409.shp"))

In [ ]:
gdf_sheds = gdf_sheds.rename(columns={"lake":"lake_name"})

In [ ]:
lake_shapes = gpd.read_file(os.path.join(home_dir, "bv/lacsSentinelles_lakes_202409.shp"))

In [ ]:
def get_sub_lakes(lake_name):
    sub_L = len(lake_name.split("_L"))
    sub_B = len(lake_name.split("_B"))
    return (sub_L > 1) or (sub_B >1)

In [ ]:
# Remove sub lakes to still account for them in the wahter shed
lake_shapes_no_sub = lake_shapes[~lake_shapes.apply(lambda row: get_sub_lakes(row["lake_name"]), axis=1)]

### Process

In [ ]:
# Make sure to align the frames
gdf_sheds = gdf_sheds.sort_values("lake_name").reset_index(drop=True)
lake_shapes_no_sub = lake_shapes_no_sub.sort_values("lake_name").reset_index(drop=True)

In [ ]:
# Make the difference with lake shapes -> return GeoSeries
sheds_no_lakes = gdf_sheds.symmetric_difference(lake_shapes_no_sub, align=True) # symmetric to remove lake border

In [ ]:
gdf_sheds = gpd.GeoDataFrame({"lake_name":gdf_sheds.lake_name}, 
                             geometry=sheds_no_lakes, 
                             crs=gdf_sheds.crs)

In [ ]:
gdf_sheds.explore()

### Save

In [ ]:
gdf_sheds.to_file(os.path.join(home_dir, "bv/lacsSentinelles_sheds_202409.shp"))

# 3. Products

Create delivery products for each lake
* Coordinates (csv)
* Lakes (shp)
* Wateshed (shp)
* Landcover (shp)

For regional products, one needs beforehand to select the given lakes from the original shapefile (see section 3.5)

In [6]:
data_dir = "/home/aguerou/CARRTEL/lacs_sentinelles/ls_bv_lc/data/main_lakes/"

In [7]:
save_dir = "/home/aguerou/CARRTEL/lacs_sentinelles/ls_bv_lc/delivery/"

In [8]:
def format_lake_name(lake_name):
    return lake_name.replace(" ", "_").lower()

## 3.1 Coordinates

In [ ]:
coords = pd.read_csv(os.path.join(data_dir, "coords/lacsSentinelles_coords_202409.csv"))

In [ ]:
coords['lake_name'] = coords.apply(lambda row : format_lake_name(row["lake_name"]), axis=1)

In [ ]:
coords.to_csv(os.path.join(save_dir, "ancillary/lacsSentinelles_coords_202409.csv"), index=False)

## 3.2 Lakes

In [ ]:
lakes_shp = gpd.read_file(os.path.join(data_dir, "bv/lacsSentinelles_lakes_202409.shp"))

In [ ]:
lakes_shp = lakes_shp.sort_values(by="lake_name").reset_index(drop=True)

In [ ]:
lakes_shp['lake_name'] = lakes_shp.apply(lambda row : format_lake_name(row["lake_name"]), axis=1)

#### Lake by lake

In [ ]:
lake_list = list(set(coords.lake_name)) #Not to get lakes with "_Lxx"

In [ ]:
for lake in lake_list:
    
    # Create folder if not existing
    if not os.path.exists(os.path.join(save_dir, f"lake_by_lake/{lake.lower()}")):
        os.mkdir(os.path.join(save_dir, f"lake_by_lake/{lake.lower()}"))
        
    my_lake = lakes_shp.loc[lakes_shp["lake_name"].str.startswith(lake)]
    my_lake.to_file(os.path.join(save_dir, f"lake_by_lake/{lake}/lacsSentinelles_lakes_202409_{lake}.shp"))

## 3.3 Sheds

In [ ]:
sheds = gpd.read_file(os.path.join(data_dir, "bv/lacsSentinelles_sheds_202409.shp"))
sheds['lake_name'] = sheds.apply(lambda row : format_lake_name(row["lake_name"]), axis=1)

#### Lake by lake

In [ ]:
lake_list = list(set(sheds.lake_name))

In [ ]:
for lake in lake_list:
    
    # Create folder if not existing
    if not os.path.exists(os.path.join(save_dir, f"lake_by_lake/{lake.lower()}")):
        os.mkdir(os.path.join(save_dir, f"lake_by_lake/{lake.lower()}"))
        
    my_shed = sheds.loc[sheds["lake_name"].str.startswith(lake)]
    my_shed.to_file(os.path.join(save_dir, f"lake_by_lake/{lake}/lacsSentinelles_sheds_202409_{lake}.shp"))

## 3.4 Landcover

In [10]:
lc_s2_alps = rioxr.open_rasterio(os.path.join(data_dir, "../landcover/carto_landcover_s2_alps_fr.tif"), mask_and_scale=True)
lc_s2_corse = rioxr.open_rasterio(os.path.join(data_dir, "../landcover/carto_landcover_s2_corse.tif"), mask_and_scale=True)
lc_s2_pyr = rioxr.open_rasterio(os.path.join(data_dir, "../landcover/carto_landcover_s2_pyr.tif"), mask_and_scale=True)

In [12]:
gdf_bv = gpd.read_file(os.path.join(data_dir, "bv/bv_convexHull.shp"))
gdf_bv['lake_name'] = gdf_bv.apply(lambda row : format_lake_name(row["lake_name"]), axis=1)
gdf_bv = gdf_bv.sort_values(by="lake_name").reset_index(drop=True)

### 3.4.1 Clip + reprojection 

In [13]:
lc_s2_alps = lc_s2_alps.rio.reproject(gdf_bv.crs).rio.clip(gdf_bv.geometry)
lc_s2_corse = lc_s2_corse.rio.reproject(gdf_bv.crs).rio.clip(gdf_bv.geometry)
lc_s2_pyr = lc_s2_pyr.rio.reproject(gdf_bv.crs).rio.clip(gdf_bv.geometry)

### 3.4.2 Compute & save TMP

#### Rasters

In [ ]:
from rioxarray.exceptions import NoDataInBounds

In [17]:
def compute_lc_rst(lc_alps, lc_corse, lc_pyr, sheds_shp):
    
    for index, row in sheds_shp.iterrows(): 

        # Create folder if not existing
        if not os.path.exists(os.path.join(save_dir, f"lake_by_lake/tmp")):
            os.mkdir(os.path.join(save_dir, f"lake_by_lake/tmp"))

        if not os.path.exists(os.path.join(save_dir, f"lake_by_lake/tmp/{row.lake_name}")):
            os.mkdir(os.path.join(save_dir, f"lake_by_lake/tmp/{row.lake_name}"))
            
        print(row.lake_name)
            
        # Try to clip from one regional raster 
        try: 
            lake_raster = lc_alps.rio.clip([row.geometry], crs=sheds_shp.crs)
        except NoDataInBounds:
            try:
                lake_raster = lc_corse.rio.clip([row.geometry], crs=sheds_shp.crs)
            except NoDataInBounds:
                lake_raster = lc_pyr.rio.clip([row.geometry], crs=sheds_shp.crs)

        # Save raster       
        lake_raster.rio.to_raster(os.path.join(save_dir, f"lake_by_lake/tmp/{row.lake_name}/lacsSentinelles_landcoverS2_2017_2023_{row.lake_name}.tif"))

In [18]:
compute_lc_rst(lc_s2_alps, lc_s2_pyr, lc_s2_corse, gdf_bv)

anterne


#### Vectors

In [19]:
def gdal_vectorizing(file, lake_by_region=False):

    qgis_dir = "/home/aguerou/miniconda3/envs/py311/bin"
    
    if lake_by_region:
        shp_file = (f"{file.split('.')[0]}.shp").replace("raster", "vector")
    else:
        shp_file = f"{file.split('.')[0]}.shp"
    
    if os.path.exists(shp_file):
        os.remove(shp_file)
        
    gdal_cmd = f"{qgis_dir}/gdal_polygonize.py\
        {file}\
        -b 1\
        -f 'ESRI Shapefile' {shp_file}\
        {os.path.basename(file.split('.')[0])} LC"
        
    os.system(gdal_cmd)

In [20]:
raster_list = glob.glob(os.path.join(save_dir, "lake_by_lake/tmp/*/*.tif"))
for raster in raster_list:
    gdal_vectorizing(raster)

0...10...20...30...40...50...60...70...80...90...Creating output /home/aguerou/CARRTEL/lacs_sentinelles/ls_bv_lc/delivery/lake_by_lake/tmp/anterne/lacsSentinelles_landcoverS2_2017_2023_anterne.shp of format ESRI Shapefile.
100 - done.


### 3.4.3 Clip to sheds + add lakes to shp + final save

In [21]:
sheds_shp = gpd.read_file(os.path.join(data_dir, "bv/lacsSentinelles_sheds_202409.shp"))
sheds_shp['lake_name'] = sheds_shp.apply(lambda row : format_lake_name(row["lake_name"]), axis=1)

/home/aguerou/miniconda3/envs/py311/lib/python3.11/site-packages/pyogrio/raw.py:194: RuntimeWarning: /home/aguerou/CARRTEL/lacs_sentinelles/ls_bv_lc/data/main_lakes/bv/lacsSentinelles_sheds_202409.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


In [22]:
lakes_shp = gpd.read_file(os.path.join(data_dir, "bv/lacsSentinelles_lakes_202409.shp"))
lakes_shp['lake_name'] = lakes_shp.apply(lambda row : format_lake_name(row["lake_name"]), axis=1)

In [23]:
from pyce import shape

In [28]:
for lake_name in sheds_shp["lake_name"]:

    print(lake_name)
    
    # Select data for given lake
    shed_lake = sheds_shp.loc[sheds_shp.lake_name==lake_name]
    water_lake = lakes_shp.loc[lakes_shp.lake_name.str.startswith(lake_name)]
    water_lake = water_lake.assign(LC=5)
    lc_lake = gpd.read_file(os.path.join(save_dir, f'lake_by_lake/tmp/{lake_name}/lacsSentinelles_landcoverS2_2017_2023_{lake_name}.shp'))
    
    # Remove lake region
    lc_lake_shp_nolake = lc_lake.overlay(water_lake, how="difference")

    # Replace by lake value (LC=5) 
    lc_shed = pd.concat([lc_lake_shp_nolake, water_lake]).reset_index(drop=True).drop(columns=["lake_name"])
    
    # Clip to sheds / at the end so the main lake is removed from the landcover
    lc_shed = lc_shed.clip(shed_lake)
    lc_shed = shape.remove_linestring(lc_shed)
    
    # Save
    lc_shed.to_file(os.path.join(save_dir, f'lake_by_lake/{lake_name}/lacsSentinelles_landcoverS2_2017_2023_{lake_name}.shp'))

anterne


### 3.4.5 Remove tmp folder

In [29]:
os.system(f"rm -r {os.path.join(save_dir, f'lake_by_lake/tmp/')}")

0

## 3.5 Regional products

In [30]:
regions = ["alps", "corse", "pyr"]
products = ["sheds", "lakes", "landcover"]

In [31]:
lake_alps = [os.path.basename(lk_path) for lk_path in glob.glob(os.path.join(save_dir, "lake_by_lake/*"))]
lake_alps.sort()
lake_pyr = ["aumar", "izourt"]
lake_corse = ["capitello", "melo", "nino"]

In [34]:
for prod in products: 
    
    for reg, lake_list in zip(regions, [lake_alps, lake_corse, lake_pyr]):
        shp_prod_reg = pd.DataFrame()
        
        for lake in lake_list:
            shp_lake = gpd.read_file(glob.glob(os.path.join(save_dir, f"lake_by_lake/{lake}/lacsSentinelles_{prod}*_{lake}.shp"))[0])
            shp_prod_reg = pd.concat([shp_prod_reg, shp_lake])

        shp_prod_reg.to_file(os.path.join(save_dir, f"lakes_by_region/{prod}/lacsSentinelles_{prod}_{reg}.shp"))

/home/aguerou/miniconda3/envs/py311/lib/python3.11/site-packages/pyogrio/raw.py:194: RuntimeWarning: /home/aguerou/CARRTEL/lacs_sentinelles/ls_bv_lc/delivery/lake_by_lake/noir_du_carro/lacsSentinelles_sheds_202409_noir_du_carro.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


# 4. Landcover statistics + plots

In [35]:
from pyce.tools.lc_mapping import LandCoverMap
from pyce_scripts import lacs_sentinelles_plots_stats

## 4.1 parameters

In [ ]:
data_dir = "/home/aguerou/CARRTEL/lacs_sentinelles/ls_bv_lc/main_lakes/data/"

In [ ]:
save_dir = "/home/aguerou/CARRTEL/lacs_sentinelles/ls_bv_lc/delivery/"

LC map

In [ ]:
mapping_file = os.path.join("/home/aguerou/data/land_cover/item/carto_h1b/nomenclature_h1b.txt")
mapping_name = "h1b_paper"

In [ ]:
lcmap = LandCoverMap(mapping_file, name=mapping_name)
lcmap = lcmap.remove_item(col_name="Code", col_val=[8, 9])
lcmap.reindex_from_col_val(col_name="Code", values=[0, 1, 6, 2, 3, 4, 5], in_place=True)

List lakes

In [ ]:
coords_lakes = pd.read_csv(os.path.join(save_dir, "ancillary/lacsSentinelles_coords_202409.csv"))

In [ ]:
list_lakes = list(set(coords_lakes.lake_name))

## 4.2 Create CSV table

### Surface

In [ ]:
stats_surf = lacs_sentinelles_plots_stats.get_stats_surface(lakes_list=list_lakes,
                                                            data_dir=data_dir,
                                                            save_dir=save_dir,
                                                            lcmap=lcmap,
                                                            save=True)

### Percent

In [ ]:
stats_perc = lacs_sentinelles_plots_stats.get_stats_percent(stat_shp=stats_surf,
                                                            lcmap=lcmap,
                                                            save_dir=save_dir)

## 4.3 Plots

In [ ]:
lcmap_map = lcmap.reindex_from_list(range(len(lcmap.df.index)))
lcmap_map.df = lcmap_map.df.sort_values("Code")

In [ ]:
lcmap.df.loc[5,"Type"] = "glacier"
lcmap.df.loc[6,"Type"] = "lakes"

In [ ]:
lcmap_map = lcmap.reindex_from_list(range(len(lcmap.df.index)))
lcmap_map.df = lcmap_map.df.sort_values("Code")

In [ ]:
reload(lacs_sentinelles_plots_stats)

In [ ]:
lacs_sentinelles_plots_stats.plot_landcover(stat_file=os.path.join(save_dir, "statistics/lacsSentineslles_statistics.csv"),
                                            lcmap=lcmap, 
                                            data_dir=save_dir,
                                            save_fig=save_dir)